In [3]:
import warnings
import igraph as ig
import networkx as nx
import collections
import numpy as np
import scipy as sp
#import bMatching as bm
#import  Evaluation as ev
import copy
import importlib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import matplotlib.pylab as plt
import pickle
import math
import pandas as pd

In [4]:
def read_networks(f1,f2,fl=None,input_dir=None):
    
    if input_dir is not None:
        f1=input_dir+f1
        f2=input_dir+f2
        
    T=nx.read_leda(f1)
    nx.write_graphml(T,'graph.graphml')
    G1 = ig.read('graph.graphml',format="graphml")
    
    T=nx.read_leda(f2)
    nx.write_graphml(T,'graph.graphml')
    G2 = ig.read('graph.graphml',format="graphml")
    
    L=None
    if fl is not None:
        if input_dir is not None:
            fl=input_dir+fl
        L=ig.read(fl,format="graphml")
    
    return G1,G2,L

In [5]:
f=open('embc.txt')

data={}
for line in f:
    line=line.strip('\n')
    line=line.split('$')
    
    tag=line[0]
    
    tag=tag.split('/')
    g=str(tag[2])
    
    #p=str(tag[3])  ### Mikel
    p=str(line[1]) ### Edoardo
    
    try:
        t=data[g]
    except:
        data[g]={}
        t=data[g]
    
    try:
        t=data[g][p]
    except:
        data[g][p]=[]
        t=data[g][p]
    
    #data[g][p]=np.array(line[1:],dtype=float)  # mikel
    data[g][p]=np.array(line[2:],dtype=float)  # Edoardo
    
f.close()

pickle.dump(data,open('structural_cone.p','wb'))

In [6]:
data=pickle.load(open('structural_cone.p','rb'))

In [11]:
input_dir="/home/khan242/netAlign/data/synthetic_networks/"
#input_dir="/Users/khan242/PNNL/netAlign/data/real_world_networks/"
f1="yeast0_Y2H1.gw"
f2="yeast5_Y2H1.gw"
p1=(f1.split('.')[0]).split('_')[0]
p2=(f2.split('.')[0]).split('_')[0]
kfactor=1
Lf=input_dir+'L_'+str(kfactor)+'_'+p1+'_'+p2+'.graphml'
print(Lf)

G1=data[f1]
G2=data[f2]

feature1=np.array([G1[key] for key in G1.keys()])
feature2=np.array([G2[key] for key in G2.keys()])
print(type(feature1),feature1.shape)

K1=int(len(G1.keys())*kfactor)
K2=int(len(G2.keys())*kfactor)
print('K values: ',K1,K2)

nbrs1 = NearestNeighbors(n_neighbors=K1,metric='cosine').fit(feature1[:,1:].astype(float))
#nbrs1 = NearestNeighbors(n_neighbors=K1,algorithm='brute').fit(feature1[:,1:].astype(float))
distances1, indices1 = nbrs1.kneighbors(feature2[:,1:].astype(float))

nbrs2 = NearestNeighbors(n_neighbors=K2,metric='cosine').fit(feature2[:,1:].astype(float))
#nbrs2 = NearestNeighbors(n_neighbors=K2,algorithm='brute').fit(feature2[:,1:].astype(float))
distances2, indices2 = nbrs2.kneighbors(feature1[:,1:].astype(float))

L={}

n1=list(G1.keys())
n2=list(G2.keys())

#print(n1)

for i in range(len(n1)):
    for jj in range(K2):
        j=indices2[i][jj]
        w=distances2[i][jj]
        if math.isnan(w):
            continue
        L[(n1[i],n2[j])]=[w]  #### Converting it to similarity
        #L[(n1[i],n2[j])]=[w]

for i in range(len(n2)):
    for jj in range(K1):
        j=indices1[i][jj]
        w=distances1[i][jj]
        if math.isnan(w):
            continue
        try:
            L[(n1[j],n2[i])].append(w )
            #L[(n1[j],n2[i])].append(w)
        except:
            pw=w ### Just ignoring !!

remove_keys=[]
for k,v in L.items():
#     if k[0]=='PAP1':
#         print(k,L[k])
    if len(v)== 2:
        val=(v[0]+v[1])*0.5
        val=1/(math.exp(val))*1000  ### 1/sqrt(e^w1+e^w2)
        L[k]=val
    else:
        remove_keys.append(k)
for k in remove_keys:
    del L[k]


# s='CFT2'
# s='PAP1'
# s='YSH1'
# for k in L.keys():
#     if k[0]==s:
#         print(k,L[k])

G1,G2,_=read_networks(f1,f2,input_dir=input_dir)

n1=G1.vcount()
n2=G2.vcount()
G1_ids={}
G2_ids={}

for i in range(n1):
    G1_ids[G1.vs[i]['id']]=i

for i in range(n2):
    G2_ids[G2.vs[i]['id']]=i+n1



GL=ig.Graph()
GL.add_vertices(list(range(n1+n2)))
edges=[]
weights=[]
for (u,v) in L.keys():
    
    i=G1_ids[u]
    j=G2_ids[v]
    w=L[(u,v)]
    edges.append((i,j))
    weights.append(w)
    
#     if u=='PAP1':
#         print(len(edges),len(weights))
#         print(u,v)
#         print(i,j)
#         print(w)
#         print()

# print(edges[0],weights[0])

GL.add_edges(edges)
GL.es['weight']=weights
print(GL.vcount(),GL.ecount(),GL.is_directed())

ig.Graph.write_graphml(GL,Lf)
print("done")

/home/khan242/netAlign/data/synthetic_networks/L_1_yeast0_yeast5.graphml
<class 'numpy.ndarray'> (1004, 150)
K values:  1004 1004
2008 1008016 False
done
